 # PtyRAD - PTYchographic Reconstruction with Automatic Differentiation

 Chia-Hao Lee

cl2696@cornell.edu

Created 2024.03.08

# 01. Imports

In [ ]:
%reload_ext autoreload
%autoreload 2

import os
import matplotlib.pyplot as plt

import numpy as np
import torch
from tifffile import imread, imwrite

GPUID = 0
DEVICE = torch.device('cuda:' + str(GPUID))
print('Execution device: ', DEVICE)
print('PyTorch version: ', torch.__version__)
print('CUDA available: ', torch.cuda.is_available())
print('CUDA version: ', torch.version.cuda)
print('CUDA device:', torch.cuda.get_device_name(GPUID))

In [ ]:
from ptyrad.initialization import Initializer
from ptyrad.models import PtychoAD
from ptyrad.optimization import CombinedLoss, CombinedConstraint, ptycho_recon, loss_logger
from ptyrad.visualization import plot_forward_pass
from ptyrad.utils import test_loss_fn, make_batches, make_save_dict, make_recon_params_dict, shuffle_batches, get_size_bytes, time_sync

# 02. Initialize optimization

In [ ]:
#from ptyrad.demos.params_CNS import exp_params, source_params
#from ptyrad.demos.params_PSO_128 import exp_params, source_params
#from ptyrad.demos.params_PSO_256 import exp_params, source_params
#from ptyrad.demos.params_tBL_WSe2 import exp_params, source_params
from ptyrad.demos.params_BaM_128 import exp_params, source_params

In [ ]:

init = Initializer(exp_params, source_params).init_all()

In [ ]:
print(init.init_variables['crop_pos'][0])

In [ ]:
model_params = {
    'init_variables': init.init_variables,
    'lr_params':{
        'obja': 5e-4,
        'objp': 5e-4,
        'probe': 5e-4, 
        'probe_pos_shifts': 0}}

model = PtychoAD(model_params, device=DEVICE)

# Use model.set_optimizer(new_lr_params) to update the variable flag and optimizer_params
optimizer = torch.optim.Adam(model.optimizer_params)

In [ ]:
# # Use this to edit learning rate if needed some refinement

# model.set_optimizer(lr_params={'obja': 1e-4,
#                                'objp': 1e-4,
#                                'probe': 1e-4, 
#                                'probe_pos_shifts': 1e-4})
# optimizer=torch.optim.Adam(model.optimizer_params)

## Check the forward pass

In [ ]:
indices = np.random.randint(0,exp_params['N_scans'],2)
dp_power = 0.5

plot_forward_pass(model, indices, dp_power, init.init_variables['obj'])

## Finetune the loss params

In [ ]:
loss_params = {
    'loss_single': {'state':  True,  'weight': 1.0, 'dp_pow': 0.5},
    'loss_pacbed': {'state': False,  'weight': 1.0, 'dp_pow': 0.2},
    'loss_tv'    : {'state': False,  'weight': 1e-5},
    'loss_l1'    : {'state': False,   'weight': 0.1},
    'loss_l2'    : {'state': False,  'weight': 1.0},
    'loss_postiv': {'state':  False,  'weight': 1.0}
}

indices = np.random.randint(0,exp_params['N_scans'],64)
loss_fn = CombinedLoss(loss_params, device=DEVICE)
test_loss_fn(model, indices, loss_fn)

# Setup the iteration-wise constraint params

In [ ]:
constraint_params = {
    'ortho_pmode'   : {'freq': 1},
    'ortho_omode'   : {'freq': None},
    'kz_filter'     : {'freq': 1, 'beta':1, 'alpha':1, 'z_pad':None},
    'postiv'        : {'freq': 1},
    'fix_probe_int' : {'freq': 1}
}

constraint_fn = CombinedConstraint(constraint_params, device=DEVICE)

# 03. Main optimization loop

In [ ]:
NITER = 5
BATCH_SIZE = 256
GROUP = 'random' #'sparse'


indices = np.arange(model.measurements.size(0))
pos = model.crop_pos.cpu().numpy()
batches = make_batches(indices, pos, BATCH_SIZE, group = GROUP)

output_path = f"output/BaM/{model.opt_objp.size(0)}obj_{model.opt_objp.size(1)}slice_N{len(indices)}_dp{model.measurements.size(1)}_lr{model_params['lr_params']['objp']:.0e}_b{BATCH_SIZE}_{GROUP}/"
print(f"output_path = {output_path}")
os.makedirs(output_path, exist_ok=True)
recon_params = make_recon_params_dict(NITER, BATCH_SIZE, GROUP, batches, output_path)

loss_iters = []
for iter in range(1, NITER+1):
    
    batches = shuffle_batches(batches, BATCH_SIZE, GROUP)
    batch_losses, iter_t = ptycho_recon(batches, model, optimizer, loss_fn, constraint_fn, iter)
    loss_iters.append((iter, loss_logger(batch_losses, iter, iter_t)))
    
    ## Saving
    if iter % 5 == 0:
        save_dict = make_save_dict(model, exp_params, source_params, loss_params, constraint_params, recon_params, loss_iters, iter_t, iter, batch_losses)
        torch.save(save_dict, os.path.join(output_path, f"model_iter{str(iter).zfill(4)}.pt"))
        imwrite(os.path.join(output_path, f"objp_iter{str(iter).zfill(4)}.tif"), model.opt_objp[0].detach().cpu().numpy().astype('float32'))
        imwrite(os.path.join(output_path, f"probe_amp_iter{str(iter).zfill(4)}.tif"), model.opt_probe.reshape(-1, model.opt_probe.size(-1)).t().abs().detach().cpu().numpy().astype('float32'))

In [ ]:
# Quick plot of the loss curve
plt.figure()
plt.plot(np.array(loss_iters)[:,1])
plt.show()

In [ ]:
# Comparing the initial probe intensity, CBED intensity, and optimized probe int
init_probe_int = np.sum(np.abs(init.init_variables['probe'])**2)
init_CBED_int = np.sum(np.mean(init.init_variables['measurements'], 0))
opt_probe_int = model.opt_probe.abs().pow(2).sum().detach().cpu().numpy()
print(f"{init_probe_int}, {init_CBED_int}, {opt_probe_int}")

In [ ]:
# Visualize the probe modes
init_probe = init.init_variables['probe']
opt_probe = model.opt_probe.detach().cpu().numpy()
fig, axs = plt.subplots(2, len(opt_probe), figsize=(len(opt_probe)*2.5, 6))
for i in range(len(opt_probe)):
    ax_init = axs[0, i]
    ax_init.set_title(f"Init probe {i}")
    im_init = ax_init.imshow(np.abs(init_probe[i]))
    ax_init.axis('off')
    plt.colorbar(im_init, ax=ax_init, shrink=0.6)

    ax_opt = axs[1, i]
    ax_opt.set_title(f"Opt probe {i}")
    im_opt = ax_opt.imshow(np.abs(opt_probe[i]))
    ax_opt.axis('off')
    plt.colorbar(im_opt, ax=ax_opt, shrink=0.6)

plt.tight_layout()
plt.show()